In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

import numpy as np

import os

# import matplotlib.pyplot as plt

# Loading Dataset

By using pytorch dataloader, files in dataset are loaded and transformed just in time.  This makes the classifier more memmory efficient.

In [36]:
DATASET_FOLDER = './training/'

# timeChannel = 1; time included
# dataFormat: 1 = double; 2 = float; 3 = short

adibin_header_dtype = np.dtype([('magic', np.string_, 4), 
                                ('version', np.int32, 1), 
                                ('secPerTick', np.float64, 1), 
                                ('year', np.int32, 1), 
                                ('month', np.int32, 1), 
                                ('day', np.int32, 1), 
                                ('hour', np.int32, 1), 
                                ('min', np.int32, 1), 
                                ('sec', np.float64, 1), 
                                ('trigger', np.float64, 1), 
                                ('numChannels', np.int32, 1), 
                                ('samplesPerChannel', np.int32, 1), 
                                ('timeChannel', np.int32, 1), 
                                ('dataFormat', np.int32, 1)])

print(adibin_header_dtype.itemsize)

channel_header_dtype = np.dtype([('channelTitle', np.string_, 32), 
                                 ('units', np.string_, 32), 
                                 ('scale', np.float64, 1), 
                                 ('offset', np.float64, 1), 
                                 ('rangeHight', np.float64, 1),
                                 ('rangeLow', np.float64, 1)])

print(channel_header_dtype.itemsize)

68
96


In [49]:
class afibDataset(Dataset):
    __xs = []
    __ys = []
    
    def __init__(self, folder_dataset, transform=None):
        self.transform = transform
        # Open and load text file including the whole training data
        with open(folder_dataset + "data.txt") as f:
            for line in f:
                # Image path
                self.__xs.append(folder_dataset + line.split()[0])        
                # Steering wheel label
                self.__ys.append(np.float32(line.split()[1]))
                
    def __getitem__(self, index):
        
        data = None
        
        with open(self.__xs[index], 'rb') as file:
            
            header_buff = file.read(68)
            header = np.frombuffer(header_buff, dtype=adibin_header_dtype)
            
            dataFormat = header[0]['dataFormat']
            numChannels = header[0]['numChannels']
            
            chan_buff = file.read(96*numChannels)
            chan_header = np.frombuffer(chan_buff, dtype=channel_header_dtype)

            if dataFormat == 1:
                frames = np.frombuffer(file.read(), dtype=np.int64)
            elif dataFormat == 2:
                frames = np.frombuffer(file.read(), dtype=np.int32)
            elif dataFormat == 3:
                frames = np.frombuffer(file.read(), dtype=np.int16)
            else:
                print('Invalid channel data format')
                return None, None
            
            deinterleaved = [frames[idx::numChannels] for idx in range(numChannels)]
            
            print(deinterleaved[0])
        
        if self.transform is not None:
            data = self.transform(data)

        # Convert image and label to torch tensors
#         img = torch.from_numpy(np.asarray(img))
        label = torch.from_numpy(np.asarray(self.__ys[index]).reshape([1,1]))
        return data, label

    # Override to give PyTorch size of dataset
    def __len__(self):
        return len(self.__xs)

In [50]:
dataset = afibDataset(DATASET_FOLDER)

data = dataset.__getitem__(0)

[array([-1124306162, -1124523907, -1124775819, ...,  1055355835,
        1055314678,  1055272777], dtype=int32)]


In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

In [ ]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

In [ ]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

In [ ]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [ ]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

In [ ]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

In [ ]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update